<a href="https://colab.research.google.com/github/lizsolano/Optativa_profesionalizante_I/blob/main/NYHouseDatabase.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt

In [5]:
#Cargamos la bd

#data = pd.read_csv("C:/Users/Lizeth Solano Romo/OneDrive - Universidad Autónoma de Aguascalientes/2024/MATERIAS/agosto-diciembre/LITC/7o/Ejercicios Phyton/NY-House-Dataset.csv")
data = pd.read_csv(r"C:\Users\lizet\OneDrive - Universidad Autónoma de Aguascalientes\2024\MATERIAS\agosto-diciembre\LITC\7o\Ejercicios Phyton\NY-House-Dataset.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\lizet\\OneDrive - Universidad Autónoma de Aguascalientes\\2024\\MATERIAS\\agosto-diciembre\\LITC\\7o\\Ejercicios Phyton\\NY-House-Dataset.csv'

In [ ]:
#Cargamos la bd

data = pd.read_csv("C:/Users/Lizeth Solano Romo/OneDrive - Universidad Autónoma de Aguascalientes/2023/diplomado unam/Modulo III/practicas/NY-House-Dataset.csv")

# Realizamos el analisis exploratorio que se realizó en R:
#Conteo de categorias
print(data["BROKERTITLE"].value_counts())
print(data["TYPE"].value_counts())
print(data["STATE"].value_counts())
print(data["LOCALITY"].value_counts())
print(data["SUBLOCALITY"].value_counts())

#descripcion de variables numericas:
print(data[["PRICE","BEDS","BATH","PROPERTYSQFT"]].describe())

#transformar categorias con menos de 10 registros a una nueva:
#BROKERTITLE
BROKERTITLE_conteo = data["BROKERTITLE"].value_counts()
BROKERTITLE_conservar = BROKERTITLE_conteo[BROKERTITLE_conteo>=10].index.to_list()
data["BROKERTITLE"]=np.where(data["BROKERTITLE"].isin(BROKERTITLE_conservar),data["BROKERTITLE"],"No se especifica")

#TYPE
TYPE_conteo = data["TYPE"].value_counts()
TYPE_conservar = TYPE_conteo[TYPE_conteo>=10].index.to_list()
data["TYPE"]=np.where(data["TYPE"].isin(TYPE_conservar),data["TYPE"],"No se especifica")

#STATE Y CP
data["STATE_2"]=data["STATE"].str.extract("^([^,]+)")
data["codigo_postal"]=data["STATE"].str.extractall("(\d+)").groupby(level=0).apply(lambda x: x.iloc[-1])

#STATE_2
STATE_2_conteo = data["STATE_2"].value_counts()
STATE_2_conservar = STATE_2_conteo[STATE_2_conteo>=10].index.to_list()
data["STATE_2"]=np.where(data["STATE_2"].isin(STATE_2_conservar),data["STATE_2"],"No se especifica")

#codigo_postal
codigo_postal_conteo = data["codigo_postal"].value_counts()
codigo_postal_conservar = codigo_postal_conteo[codigo_postal_conteo>=10].index.to_list()
data["codigo_postal"]=np.where(data["codigo_postal"].isin(codigo_postal_conservar),data["codigo_postal"],"No se especifica")

#LOCALITY
LOCALITY_conteo = data["LOCALITY"].value_counts()
LOCALITY_conservar = LOCALITY_conteo[LOCALITY_conteo>=10].index.to_list()
data["LOCALITY"]=np.where(data["LOCALITY"].isin(LOCALITY_conservar),data["LOCALITY"],"No se especifica")


#SUBLOCALITY
SUBLOCALITY_conteo = data["SUBLOCALITY"].value_counts()
SUBLOCALITY_conservar = SUBLOCALITY_conteo[SUBLOCALITY_conteo>=10].index.to_list()
data["SUBLOCALITY"]=np.where(data["SUBLOCALITY"].isin(SUBLOCALITY_conservar),data["SUBLOCALITY"],"No se especifica")

#TABLA DESCRIPTIVA:
Tabla_Descriptiva_Final = data.groupby(['TYPE','STATE_2','codigo_postal','LOCALITY','SUBLOCALITY']).agg(
  CONTEO=('TYPE','count'),
  PROMEDIO_PRICE=('PRICE','mean'),
  DESVIACION_PRICE=('PRICE','std'),
  PROMEDIO_BEDS=('BEDS','mean'),
  DESVIACION_BEDS=('BEDS','std'),
  PROMEDIO_BATH=('BATH','mean'),
  DESVIACION_BATH=('BATH','std'),
  PROMEDIO_PROPERTYSQFT=('PROPERTYSQFT','mean'),
  DESVIACION_PROPERTYSQFT=('PROPERTYSQFT','std')
).reset_index()

#Modelo
## Se genera el modelo en python:

features = data[["TYPE","BEDS","BATH","PROPERTYSQFT","STATE_2","codigo_postal",
"SUBLOCALITY","LOCALITY"]]

#se convierte en logaritmica la variable price
target = np.log(data["PRICE"])

#Convertir variables en dummy para el modelo:(las categoricas)
data_encoded = pd.get_dummies(features,
columns=["TYPE","STATE_2","LOCALITY","SUBLOCALITY","codigo_postal"],
drop_first=True)

X_train, X_test, y_train, y_test = train_test_split(data_encoded,target,
test_size=0.20)

model=LinearRegression()
model.fit(X_train,y_train)
predictions=model.predict(X_test)

#evaluamos el modelo
mean_absolute_error(predictions,y_test)
np.sqrt(mean_squared_error(predictions, y_test))

model.score(X_test,y_test)
model.coef_
